### Generating Events

In [1]:
import random

# Event types
floatevent = 0
intevent = 1

# Generate event of random size within range
# Return array representing event
def generate_event(min_size, max_size, event_type, rand_multiplier = 10):
    event_size = random.randint(min_size, max_size)
    
    if event_type == intevent:
        return [int(random.random() * rand_multiplier) for i in range(event_size)]
    else:
        return [random.random() * rand_multiplier for i in range(event_size)]

# Return array of n events of form (time, data)
# with times separated within dt range
# with data within size range
def generate_n_events(n, dt_min, dt_max, min_size, max_size, event_type):
    t = 0
    events = []
    for i in range(n):
        events.append((t, generate_event(min_size, max_size, event_type)))
        dt = dt_min + (random.random() * (dt_max - dt_min))
        t += dt
    return events
generate_n_events(3, 0.5, 0.8, 4, 8, floatevent)

[(0,
  [1.7771781342830328,
   0.5730881352514672,
   2.731541231267655,
   5.40876848711484]),
 (0.6465032066062002,
  [8.097078459747282,
   7.284244469315246,
   3.2933992060173134,
   0.9586089740966985,
   8.415347886512365,
   0.9007282574891717]),
 (1.3437885040522732,
  [0.9252372227085903,
   2.811573013652713,
   5.872084480876058,
   4.3141337489952125,
   8.607019657824635,
   1.4722764454144188,
   2.1155326862434976])]

### Simulation

In [7]:
import numpy as np
import time
import h5py

# Run a simulation with the given parameters
# n_events is number of events to group together to write to hdf5
def run_sim(n, group_size, dt_min, dt_max, min_size, max_size, event_type, file="out.h5"):
    #hf = h5py.File('data.h5', 'w')
    #g1 = hf.create_croup('group1')
    
    events = generate_n_events(n, dt_min, dt_max, min_size, max_size, event_type)
    
    group = []
    count = 0
    start = time.time()
    # Write the events to memory
    while(count < n):
        now = time.time() - start

        # Next event has not fired yet
        if events[count][0] > now:
            time.sleep(0.01)
            continue

        group.append(events[count][1])
        count += 1
    
    return time.time() - start

print("\n\n", run_sim(5, 2, 1, 2, 100, 200, floatevent), "s")

[[7.370303059437787, 4.879376436560204, 9.13700055002953, 9.643261163982984, 5.532578879557737, 3.492538478875762, 0.12978090462342662, 8.175347875168168, 4.428911910173071, 2.2901035138049233, 0.7814059761214376, 9.511902681258386, 1.3225148001598996, 2.089563293073291, 8.96432997182864, 8.355454729438181, 2.070495898467769, 0.05075630432229916, 9.996201309196236, 0.3577694448470825, 8.870797169279852, 3.1551706728524787, 6.653291490786952, 9.091735876682478, 7.778877190460495, 5.380740483084196, 5.573006601469014, 1.2884171378155973, 6.7816948086030235, 0.337376014877212, 7.519380715172558, 8.945719653479243, 9.09121262130329, 7.400038947635858, 8.797812391029607, 4.6883257045818025, 0.5228990260019162, 3.4470021354100346, 0.3304737602707508, 6.572897228450328, 1.1313176221438337, 3.697287856987473, 0.9106441260178444, 3.5652455120895032, 4.40076376638782, 0.4342530947975731, 4.376581591625436, 8.713839489503124, 8.201677335387918, 1.9234981968360854, 2.235341053795019, 3.53306778486

[[7.370303059437787, 4.879376436560204, 9.13700055002953, 9.643261163982984, 5.532578879557737, 3.492538478875762, 0.12978090462342662, 8.175347875168168, 4.428911910173071, 2.2901035138049233, 0.7814059761214376, 9.511902681258386, 1.3225148001598996, 2.089563293073291, 8.96432997182864, 8.355454729438181, 2.070495898467769, 0.05075630432229916, 9.996201309196236, 0.3577694448470825, 8.870797169279852, 3.1551706728524787, 6.653291490786952, 9.091735876682478, 7.778877190460495, 5.380740483084196, 5.573006601469014, 1.2884171378155973, 6.7816948086030235, 0.337376014877212, 7.519380715172558, 8.945719653479243, 9.09121262130329, 7.400038947635858, 8.797812391029607, 4.6883257045818025, 0.5228990260019162, 3.4470021354100346, 0.3304737602707508, 6.572897228450328, 1.1313176221438337, 3.697287856987473, 0.9106441260178444, 3.5652455120895032, 4.40076376638782, 0.4342530947975731, 4.376581591625436, 8.713839489503124, 8.201677335387918, 1.9234981968360854, 2.235341053795019, 3.53306778486

[[7.370303059437787, 4.879376436560204, 9.13700055002953, 9.643261163982984, 5.532578879557737, 3.492538478875762, 0.12978090462342662, 8.175347875168168, 4.428911910173071, 2.2901035138049233, 0.7814059761214376, 9.511902681258386, 1.3225148001598996, 2.089563293073291, 8.96432997182864, 8.355454729438181, 2.070495898467769, 0.05075630432229916, 9.996201309196236, 0.3577694448470825, 8.870797169279852, 3.1551706728524787, 6.653291490786952, 9.091735876682478, 7.778877190460495, 5.380740483084196, 5.573006601469014, 1.2884171378155973, 6.7816948086030235, 0.337376014877212, 7.519380715172558, 8.945719653479243, 9.09121262130329, 7.400038947635858, 8.797812391029607, 4.6883257045818025, 0.5228990260019162, 3.4470021354100346, 0.3304737602707508, 6.572897228450328, 1.1313176221438337, 3.697287856987473, 0.9106441260178444, 3.5652455120895032, 4.40076376638782, 0.4342530947975731, 4.376581591625436, 8.713839489503124, 8.201677335387918, 1.9234981968360854, 2.235341053795019, 3.53306778486